In [17]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [18]:
citrus_data = pd.read_csv("./citrus.csv", delimiter=',')

In [19]:
citrus_data.shape

(10000, 6)

In [20]:
citrus_data

,name,diameter,weight,red,green,blue
0,orange,2.96,86.76,172,85,2
1,orange,3.91,88.05,166,78,3
2,orange,4.42,95.17,156,81,2
3,orange,4.47,95.60,163,81,4
4,orange,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,grapefruit,15.35,253.89,149,77,20
9996,grapefruit,15.41,254.67,148,68,7
9997,grapefruit,15.59,256.50,168,82,20
9998,grapefruit,15.92,260.14,142,72,11


## Label Encoding

In [22]:
label_encoder = preprocessing.LabelEncoder()

In [23]:
citrus_data['name']= label_encoder.fit_transform(citrus_data['name'])

In [24]:
citrus_data

,name,diameter,weight,red,green,blue
0,1,2.96,86.76,172,85,2
1,1,3.91,88.05,166,78,3
2,1,4.42,95.17,156,81,2
3,1,4.47,95.60,163,81,4
4,1,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,0,15.35,253.89,149,77,20
9996,0,15.41,254.67,148,68,7
9997,0,15.59,256.50,168,82,20
9998,0,15.92,260.14,142,72,11


In [25]:
citrus_data.columns

Index(['name', 'diameter', 'weight', 'red', 'green', 'blue'], dtype='object')

In [26]:
columns = ['diameter', 'weight', 'red', 'green', 'blue']

In [27]:
X = citrus_data[columns].to_numpy()
y = citrus_data['name'].to_numpy()

In [28]:
print(X)

[[  2.96  86.76 172.    85.     2.  ]
 [  3.91  88.05 166.    78.     3.  ]
 [  4.42  95.17 156.    81.     2.  ]
 ...
 [ 15.59 256.5  168.    82.    20.  ]
 [ 15.92 260.14 142.    72.    11.  ]
 [ 16.45 261.51 152.    74.     2.  ]]


In [29]:
print(y)

[1 1 1 ... 0 0 0]


In [34]:
class NeuralNetwork(object):
    def __init__(self, inputSize, outputSize, hiddenSize, hiddenLayers, epochs=1000, learningRate=0.01):
        # parameters
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenSize = hiddenSize
        self.hiddenLayers = hiddenLayers
        self.epochs = epochs
        self.learningRate = learningRate
        self.W = [0] * (hiddenLayers+1)
                
        # weights initialization
        for i in range(self.hiddenLayers):
            if(i==0):
                self.W[i] = np.random.randn(self.inputSize, self.hiddenSize[i])
            else:
                self.W[i] = np.random.rand(self.hiddenSize[i-1], self.hiddenSize[i])
        
        self.W[self.hiddenLayers] = np.random.rand(self.hiddenSize[self.hiddenLayers-1], self.outputSize)
        
        #for i in range(len(self.W)):
        #    print("W[" + str(i) + "]")
        #    print(self.W[i])
        #    
        #print("\n")
        
    
    def sigmoid(self, s, deriv=False):
        if(deriv == True):
            return s*(1 - s)
        return 1/(1 + np.exp(-s))
    
    
    def feedForward(self, X):
        self.Z = []
        
        for i in range(self.hiddenLayers + 1):
            if(i==0):
                self.Z.append(self.sigmoid(np.dot(X, self.W[i])))
            else:
                self.Z.append(self.sigmoid(np.dot(self.Z[i-1], self.W[i])))
            
        #for i in range(len(self.Z)):
        #    print("Z[" + str(i) + "]")
        #    print(self.Z[i])
        #    
        #print("\n")
        
        return self.Z[self.hiddenLayers]
    
    
    def backward(self, X, y):
        self.error = [0] * (self.hiddenLayers+1)
        self.delta = [0] * (self.hiddenLayers+1)        

        for i in range(self.hiddenLayers, -1, -1):
            if(i==self.hiddenLayers):
                self.error[i] = y - self.Z[i]
                self.delta[i] = self.error[i] * self.sigmoid(self.Z[i], deriv=True)
            else:
                self.error[i] = np.dot(self.delta[i+1], self.W[i+1].T)
                self.delta[i] = self.error[i] * self.sigmoid(self.Z[i], deriv=True)
            
        
        
        for i in range(self.hiddenLayers + 1):
            if(i==0):
                self.W[i] = self.W[i] + self.learningRate * np.dot(X.reshape(1,self.inputSize).T, self.Z[i].reshape(1,self.hiddenSize[i]))
            else:
                self.W[i] = self.W[i] + self.learningRate * np.dot(self.Z[i].T, self.delta[i])
                    
        
    def train(self, X, y):
        for i in range(self.epochs):
            for j in range(len(X)):
                self.feedForward(X[j])
                self.backward(X[j], y[j])
                
    def printWeights():
        for i in range(len(self.W)):
           print("W[" + str(i) + "]")
           print(self.W[i])
           
        print("\n")
        

In [35]:
NN = NeuralNetwork(inputSize=5, outputSize=2, hiddenSize=[6,3], hiddenLayers=2, epochs=1500, learningRate=0.01)

NN.train(X, y)

print("Input:\n" + str(X))
print("\n")

print("Actual Output:\n" + str(y))
print("\n")

print("Predicted Output:\n" + str(NN.feedForward(X)))
print("\n")

print("Loss:\n", np.mean(np.square(y - NN.feedForward(X))))

Input:
[[  2.96  86.76 172.    85.     2.  ]
 [  3.91  88.05 166.    78.     3.  ]
 [  4.42  95.17 156.    81.     2.  ]
 ...
 [ 15.59 256.5  168.    82.    20.  ]
 [ 15.92 260.14 142.    72.    11.  ]
 [ 16.45 261.51 152.    74.     2.  ]]


Actual Output:
[1 1 1 ... 0 0 0]


Predicted Output:
[[0.44456082 0.45029599]
 [0.44456082 0.45029599]
 [0.44456082 0.45029599]
 ...
 [0.44456082 0.45029599]
 [0.44456082 0.45029599]
 [0.44456082 0.45029599]]




ValueError: operands could not be broadcast together with shapes (10000,) (10000,2) 